In [1]:
import matplotlib
matplotlib.use('TkAgg')
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import logging

In [2]:
def read_image(IMG_DIR, IMG_NAME_1, IMG_NAME_2):
    img_1 = cv2.imread(IMG_DIR + IMG_NAME_1)
    img_2 = cv2.imread(IMG_DIR + IMG_NAME_2)
    #cv2.imshow(IMG_NAME_1,img_1)
    #cv2.imshow(IMG_NAME_2,img_2)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return img_1, img_2

In [3]:
IMG_DIR = 'image_0/'
logging.basicConfig(level=logging.INFO)

ERROR:tornado.application:Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x000001482D311598>)
Traceback (most recent call last):
  File "C:\Users\qaz20\Anaconda3\lib\site-packages\tornado\ioloop.py", line 743, in _run_callback
    ret = callback()
  File "C:\Users\qaz20\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 306, in advance_eventloop
    eventloop(self)
  File "C:\Users\qaz20\Anaconda3\lib\site-packages\ipykernel\eventloops.py", line 238, in loop_tk
    app.tk.createfilehandler(stream.getsockopt(zmq.FD), READABLE, notifier)
AttributeError: '_tkinter.tkapp' object has no attribute 'createfilehandler'


In [5]:
T = np.zeros((3, 1))
R = np.eye(3)
plt.ion()
fig, ax = plt.subplots()
ax.set_xlim(0,200)
ax.set_ylim(0,200)
X,Y = [],[]
sc = ax.scatter(X,Y)
plt.draw()

for root,dirs,files in os.walk(IMG_DIR):
    for i, f in enumerate(files):
        #print(i)
        #if i == 30:
            #print('here')
        #    break
        IMG_NAME_2 = f
        if f == '000000.png':
            IMG_NAME_1 = IMG_NAME_2
            continue
        img_1, img_2 = read_image(IMG_DIR, IMG_NAME_1, IMG_NAME_2)
        logging.info(f + "is read.")
        #print(f + "is read.")
        ###SIFT###
        sift = cv2.xfeatures2d.SIFT_create()
        kp1, des1 = sift.detectAndCompute(img_1,None)
        kp2, des2 = sift.detectAndCompute(img_2,None)
        SIFT_1 = cv2.drawKeypoints(img_1, kp1, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        SIFT_2 = cv2.drawKeypoints(img_2, kp2, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        #cv2.imshow('image1',SIFT_1)
        #cv2.imshow('image2',SIFT_2)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        ###SIFT###
        #logging.info("SIFT is done.")
        ###FIND MATCHES###
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        good = []
        pts1 = []
        pts2 = []
        for i,(m,n) in enumerate(matches):
            if m.distance < 0.8*n.distance:
                good.append(m)
                pts2.append(kp2[m.trainIdx].pt)
                pts1.append(kp1[m.queryIdx].pt)
        ###FIND MATCHES###
        #logging.info("Matching is done.")
        ###FIND ESSENTIALMATRIX###
        pts1_E = np.asarray(pts1)
        pts2_E = np.asarray(pts2)
        E, mask = cv2.findEssentialMat(pts1_E, pts2_E)
        pts1_E = pts1_E[mask.ravel()==1]
        pts2_E = pts2_E[mask.ravel()==1]
        points, r, t, mask = cv2.recoverPose(E, pts1_E, pts2_E)
        ###FIND ESSENTIALMATRIX###   
        #logging.info("Essential Matrix is found.")
        ###CALCULATE R AND T###
        T += -R@t
        R = r@R
        ###CALCULATE R AND T###
        #logging.info("T and R are calculated.")
        ###PLOT###
        X.append(T[0, 0])
        Y.append(T[1, 0])
        sc.set_offsets(np.c_[X, Y])
        #sp.set_data(X,Y)
        fig.canvas.draw_idle()
        #plt.pause(0.1)
        ###PLOT###
        #print(T)
        IMG_NAME_1 = IMG_NAME_2
plt.waitforbuttonpress()

INFO:root:000001.pngis read.
INFO:root:000002.pngis read.
INFO:root:000003.pngis read.
INFO:root:000004.pngis read.
INFO:root:000005.pngis read.
INFO:root:000006.pngis read.
INFO:root:000007.pngis read.
INFO:root:000008.pngis read.
INFO:root:000009.pngis read.
INFO:root:000010.pngis read.
INFO:root:000011.pngis read.
INFO:root:000012.pngis read.
INFO:root:000013.pngis read.
INFO:root:000014.pngis read.
INFO:root:000015.pngis read.
INFO:root:000016.pngis read.
INFO:root:000017.pngis read.
INFO:root:000018.pngis read.
INFO:root:000019.pngis read.
INFO:root:000020.pngis read.
INFO:root:000021.pngis read.
INFO:root:000022.pngis read.
INFO:root:000023.pngis read.
INFO:root:000024.pngis read.
INFO:root:000025.pngis read.
INFO:root:000026.pngis read.
INFO:root:000027.pngis read.
INFO:root:000028.pngis read.
INFO:root:000029.pngis read.
INFO:root:000030.pngis read.
INFO:root:000031.pngis read.
INFO:root:000032.pngis read.
INFO:root:000033.pngis read.
INFO:root:000034.pngis read.
INFO:root:0000

TclError: can't invoke "update" command: application has been destroyed